In [1]:
# Cyberthon Data Science Training Materials
# Author: Ragul Balaji <ragulbalaji@ctf.sg>
# Dataset: Public Domain
# ALT-TAB LABS LLP (C) 2019-2022

If you're opening this locally, make sure your environment has an install of the packages from the following versions. Uncomment the following cell and run it.

In [2]:
#! pip install pandas==1.1.5 scikit-learn==1.0.2 matplotlib==3.5.1

# Part I: The Problem

Can we predict who sent a tweet? 🤔

This is a classification problem - the target variable belongs to either of the two categories, in our example, the categories are `Donald Trump` and `Justin Trudeau`.

Let's dive in and explore!

## Loading the Dataset

We will load the file `train.csv` using pandas `read.csv()` into a `Dataframe` object.

In [3]:
import pandas as pd

data = pd.read_csv("train.csv")
data.head()

,trainid,author,status
0,0,Justin Trudeau,RT @CQualtro: Félicitations #Amazon pour l'exp...
1,1,Justin Trudeau,Nous cherchons à résoudre les enjeux qui compt...
2,2,Donald J. Trump,Heading into the 12 days with great negotiatin...
3,3,Donald J. Trump,The long anticipated release of the #JFKFiles ...
4,4,Donald J. Trump,....for the Middle Class. The House and Senate...


## Target variable and Predictor(s)

We first identify that our target variable is `"author"`, and our predictor variable is `"status"`.

You can also apply stemming/lemmatization (covered earlier), or engineer new features that could be useful in model prediction (not covered).

In [4]:
# identify target and predictor
y = data['author']
X = data['status']

## Training set and validation set

### Training set

- data used to train our model
- data and labels are provided to the model, the model tune its parameters to fit the model

### Validation set

- Data used to test our model after it has been trained
- Predicted labels are compared against our true labels to compute the accuracy, to determine the model's performance.

### Why shouldn't the model be trained using the test set too?

- We want our model to generalize well on unseen data.
- Trained model gains information about the validation set and the predictions made by the model will be biased towards the validation set, resulting in overestimation of the model's performance.

Here, we specify some parameters
- `random_state=42` : for reproducible results
- `test_size=0.25` : dataset will be split into 75% training and 25% validation

In [5]:
from sklearn.model_selection import train_test_split

# split dataset
X_train, X_validation, y_train, y_validation = train_test_split(X,y,random_state=42,test_size=0.25)

Great! Now that we have split our data, are we able to train the model using training set yet? No!

## Text Vectorization

As covered earlier, machine learning models require numerical numbers as inputs. Text data unfortunately doesn't work here.😞

Let's use `CountVectorizer` to convert the tweets into numerical vectors!

We first create an instance of `CountVectorizer`.

We want the model to learn the vocabulary from the training set, so we fit the `CountVectorizer` using only the training set, not the validation set.

Then transform the training set and validation set to encode each tweet as a vector.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform/vectorize the training set
vec_train = vectorizer.fit_transform(X_train)

# Transform/Vectorize the validation set
vec_validation = vectorizer.transform(X_validation)

Confused? 😓 Here's the simple demo of `CountVectorizer` using the 2 documents mentioned in the training slides.

`doc1 = "Collab between CSIT and Alt-Tab rock"`

`doc2 = "Gonna rock CTF again"`

We fit `CountVectorizer` using `doc1` and `doc2`, so the vocabulary contain words from both documents.

In [ ]:
doc1 = "Collab between CSIT and Alt-Tab rock"
doc2 = "Gonna rock CTF again"

demo_vec = CountVectorizer()
matrix = demo_vec.fit_transform([doc1, doc2]) # learn the vocab of doc1 and doc2 and return a document term matrix

vocab = demo_vec.fit([doc1,doc2]) # learn the vocab of doc1 and doc2
matrix = vocab.transform([doc1,doc2]) #converts the documents into document term matrix - in other words, doc1 and doc2 are converted to vectors

# CountVectorizer learns the vocabulary from doc1
demo_vec.get_feature_names()

Generate the document term matrix. 

There are 2 documents and 10 unique terms, hence 2x10 matrix.

Cell specify the count of term in the document.

In [ ]:
pd.DataFrame(matrix.toarray(), columns=demo_vec.get_feature_names(), index = ['doc1','doc2'])

## Machine Learning Model

We will use the library `sklearn` due to its wide selection of models. Machine learning models in `sklearn` are objects which need to be initialized first.

Let's use what we learnt earlier - Decision 🌲!

Here, we specify some parameters
- `random_state=42` : for reproducible results
- `max_depth=8` : to limit tree depth to 8

For detailed documention, refer here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

Some useful methods
- `.fit()` : to pass in our training set to train a machine learning model
- `.predict()` : to pass in our validation set to make predictions using our model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize our machine learning model
model = DecisionTreeClassifier(random_state=42, max_depth=8)

In [ ]:
# Train the model
model.fit(vec_train,y_train) # build a decision tree classifier using the vectorized training set and training labels

In [ ]:
# Make predictions on test set
y_pred = model.predict(vec_validation)

In [ ]:
# Score the model using accuracy_score
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true = y_validation, y_pred = y_pred)

print(f"Accuracy: {acc}")

# Part II: Model Interpretability

How does the model make its predictions? Let's visualize the 🌲!

### Example: Model predicts Justin Trudeau sent the tweet
Starting at the root node (the node where the 🌲 begins), <br>
if `de` is absent in the tweet, go to the left branch, <br>
if `rt` is present in the tweet, go to the right branch, <br>
if `thank` and `hannity` and `seanhannity` are absent in the tweet, <br>
the model predicts the tweet sender as `Justin Trudeau`.

In [ ]:
# RUN THIS PART AFTER YOU ARE 
# DONE WITH PART 1

from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))
tree.plot_tree(model, fontsize=10, filled=True, class_names=['Donald', 'Justin'], feature_names=vectorizer.get_feature_names())
plt.show()

The _absence_ of `de`, _presence_ of `rt`, _absence_ of `thank` and `hannity` and `seanhannity` result in the model predicting `Justin Trudeau` as the tweet sender.

# Part III: Model Attack

Now, you are able to identify what features the model uses to make its predictions. 

Are you able to make minimal modifications to a tweet to fool the model?

We have established in Part II that `de`, `rt`, `thank`, `hannity` and `seanhannity` are features used by the model to make its predictions.

Specifically, the absence of `de`, presence of `rt`, absence of `thank` and `hannity` and `seanhannity` result in the model predicting `Justin Trudeau` as the tweet sender.

Using the same features, what would fool the model to classify `Donald Trump` as the tweet sender? If `thank` or `hannity` is present in the tweet!

Let's explore modifying a sample tweet from the validation set!

In [ ]:
sampleid = 1
truth = y_validation.iloc[sampleid]

print('tweet:', repr(X_validation.iloc[sampleid]))
print('\n   classifies as:', y_pred[sampleid])
print(' ground truth is:', truth)

In [ ]:
# we want to modify an input so that the model misclassifies

original = X_validation.iloc[sampleid]
attack1 = X_validation.iloc[sampleid] + ' thank'
attack2 = X_validation.iloc[sampleid] + ' hannity'
attack3 = X_validation.iloc[sampleid] + ' seanhannity'

print('\n original:', repr(original))
print('\n attack1:', repr(attack1))
print('\n attack2:', repr(attack2))
print('\n attack3:', repr(attack3))

Again, same concept as before - vectorize text before input to machine learning models.

In [ ]:
# vectorize the attacks
vec_attack = vectorizer.transform([original, attack1, attack2, attack3])

# feed attacks into model for prediction
attack_pred = model.predict(vec_attack)

print('\n   truth:', truth)
print('\n   original:', attack_pred[0])
print('\n   attack1:', attack_pred[1])
print('\n   attack2:', attack_pred[2])
print('\n   attack3:', attack_pred[3])

Modifying the sample tweet by adding `thank` or `hannity` fooled the model to predict `Donald Trump` instead of `Justin Trudeau` as the tweet sender.

Can you identify another modification to fool the model? 😁